In [3]:
import pandas as pd
import re

In [4]:
sh_sentiment = pd.read_excel(r"C:\Users\Kompa-KTelt056\Desktop\Data Kompa\Vinamilk\Tính 23.07.2024.xlsx", sheet_name='Sentiment')
sh_relevant = pd.read_excel(r"C:\Users\Kompa-KTelt056\Desktop\Data Kompa\Vinamilk\Tính 23.07.2024.xlsx", sheet_name='Relevant')
sh_channel = pd.read_excel(r"C:\Users\Kompa-KTelt056\Desktop\Data Kompa\Vinamilk\Tính 23.07.2024.xlsx", sheet_name='Channel')
sh_label = pd.read_excel(r"C:\Users\Kompa-KTelt056\Desktop\Data Kompa\Vinamilk\Tính 23.07.2024.xlsx", sheet_name='Label')
sh_channel.sentiment = pd.read_excel(r"C:\Users\Kompa-KTelt056\Desktop\Data Kompa\Vinamilk\Tính 23.07.2024.xlsx", sheet_name='Channel.Sentiment')

C:\Users\Kompa-KTelt056\AppData\Local\Temp\ipykernel_18280\2400443167.py:5: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  sh_channel.sentiment = pd.read_excel(r"C:\Users\Kompa-KTelt056\Desktop\Data Kompa\Vinamilk\Tính 23.07.2024.xlsx", sheet_name='Channel.Sentiment')


# Phân tích biến

Sheet Sentiment

In [3]:
sh_sentiment.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15645 entries, 0 to 15644
Data columns (total 23 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Thương hiệu       15645 non-null  object        
 1   Ngành hàng        15645 non-null  object        
 2   Sản phẩm          15645 non-null  object        
 3   Thời gian         15645 non-null  datetime64[ns]
 4   Tuần              15645 non-null  object        
 5   Positive          10673 non-null  float64       
 6   Neutral           15397 non-null  float64       
 7   Negative          5430 non-null   float64       
 8   Blank             5429 non-null   float64       
 9   Total after       15645 non-null  float64       
 10  Blank.1           15644 non-null  float64       
 11  Total             15645 non-null  float64       
 12  Buzzes thêm       330 non-null    float64       
 13  Sum of comments   11677 non-null  float64       
 14  Sum of shares     1167

In [4]:
object_columns = list(sh_sentiment.select_dtypes(['object']).columns)
object_columns

['Thương hiệu',
 'Ngành hàng',
 'Sản phẩm',
 'Tuần',
 'Ghi chú',
 'Số campaign',
 'Phân loại']

In [5]:
# Đếm số lượng giá trị duy nhất cho mỗi cột
for column in object_columns: 
    print("*, Column: ", column)   
    print(len(sh_sentiment[column].unique()), "unique values", "\t & \t", 
          sh_sentiment[column].notnull().sum(), "non-null values\t", 
          round(100* sh_sentiment[column].notnull().sum()/len(sh_sentiment[column]),2), 
          "% non-null") 

*, Column:  Thương hiệu
17 unique values 	 & 	 15645 non-null values	 100.0 % non-null
*, Column:  Ngành hàng
16 unique values 	 & 	 15645 non-null values	 100.0 % non-null
*, Column:  Sản phẩm
274 unique values 	 & 	 15645 non-null values	 100.0 % non-null
*, Column:  Tuần
89 unique values 	 & 	 15645 non-null values	 100.0 % non-null
*, Column:  Ghi chú
293 unique values 	 & 	 491 non-null values	 3.14 % non-null
*, Column:  Số campaign
107 unique values 	 & 	 175 non-null values	 1.12 % non-null
*, Column:  Phân loại
21 unique values 	 & 	 15399 non-null values	 98.43 % non-null


In [6]:
for column in object_columns:
    print('-'*100)
    print("*, Column: ", column)
    print(sh_sentiment[column].value_counts())
    print('-'*100)

----------------------------------------------------------------------------------------------------
*, Column:  Thương hiệu
Vinamilk        4108
TH True Milk    1749
Nutifood        1636
Nutricare       1388
Vitadairy       1328
Nestlé          1205
FCV              833
IDP              685
Mead Johnson     586
Abbott           534
Vinasoy          324
Đà Lạt Milk      287
Hipp             267
Fonterra         253
Yakult           178
Betagen          172
Hoff             112
Name: Thương hiệu, dtype: int64
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
*, Column:  Ngành hàng
SB       5806
SN       2710
SC       2124
BRAND    1319
NGK       938
SĐN       893
SĐ        544
KEM       513
BAD       306
BDD       189
BƠ         85
PM         68
ĐƯỜNG      61
FOOD       54
Brand      30
TUNL        5
Name: Ngành hàng, dtype: int64
------

Sheet Relevant

In [7]:
sh_relevant.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3590 entries, 0 to 3589
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Thương hiệu       3590 non-null   object        
 1   Ngành hàng        3590 non-null   object        
 2   Sản phẩm          3590 non-null   object        
 3   Thời gian         3590 non-null   datetime64[ns]
 4   Relevant          3581 non-null   float64       
 5   Irrelevant        2281 non-null   float64       
 6   Blank             1293 non-null   float64       
 7   Total after       3590 non-null   int64         
 8   Total             3590 non-null   float64       
 9   Buzzes thêm       193 non-null    float64       
 10  Relevant new      3590 non-null   float64       
 11  Irrelevant new    3590 non-null   float64       
 12  Total new         3590 non-null   float64       
 13  Check Total Buzz  3590 non-null   int64         
 14  Ghi chú           4 non-

In [8]:
object_columns = list(sh_relevant.select_dtypes(['object']).columns)
object_columns

['Thương hiệu', 'Ngành hàng', 'Sản phẩm', 'Ghi chú']

In [9]:
# Đếm số lượng giá trị duy nhất cho mỗi cột
for column in object_columns: 
    print("*, Column: ", column)   
    print(len(sh_relevant[column].unique()), "unique values", "\t & \t", 
          sh_relevant[column].notnull().sum(), "non-null values\t", 
          round(100* sh_relevant[column].notnull().sum()/len(sh_relevant[column]),2), 
          "% non-null") 

*, Column:  Thương hiệu
17 unique values 	 & 	 3590 non-null values	 100.0 % non-null
*, Column:  Ngành hàng
16 unique values 	 & 	 3590 non-null values	 100.0 % non-null
*, Column:  Sản phẩm
275 unique values 	 & 	 3590 non-null values	 100.0 % non-null
*, Column:  Ghi chú
5 unique values 	 & 	 4 non-null values	 0.11 % non-null


In [10]:
for column in object_columns:
    print('-'*100)
    print("*, Column: ", column)
    print(sh_relevant[column].value_counts())
    print('-'*100)

----------------------------------------------------------------------------------------------------
*, Column:  Thương hiệu
Vinamilk        947
Nutifood        396
Nutricare       392
TH True Milk    375
Vitadairy       290
Nestlé          268
FCV             190
IDP             152
Mead Johnson    130
Abbott          108
Vinasoy          70
Đà Lạt Milk      62
Fonterra         54
Hipp             54
Betagen          42
Yakult           36
Hoff             24
Name: Thương hiệu, dtype: int64
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
*, Column:  Ngành hàng
SB       1378
SN        588
SC        464
BRAND     285
NGK       232
SĐN       209
SĐ        124
KEM       106
BAD        72
BDD        50
ĐƯỜNG      19
BƠ         18
PM         16
Kem        12
FOOD       12
TUNL        5
Name: Ngành hàng, dtype: int64
-----------------------

Sheet Channel

In [11]:
sh_channel.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3589 entries, 0 to 3588
Data columns (total 36 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   Thương hiệu                   3589 non-null   object        
 1   Ngành hàng                    3589 non-null   object        
 2   Sản phẩm                      3589 non-null   object        
 3   Thời gian                     3589 non-null   datetime64[ns]
 4   Fanpage                       2157 non-null   float64       
 5   FB page                       3064 non-null   float64       
 6   FB user                       3124 non-null   float64       
 7   FB group                      3091 non-null   float64       
 8   Tiktok                        1820 non-null   float64       
 9   Forum                         737 non-null    float64       
 10  News                          1128 non-null   float64       
 11  Youtube                       

In [12]:
object_columns = list(sh_channel.select_dtypes(['object']).columns)
object_columns

['Thương hiệu', 'Ngành hàng', 'Sản phẩm', 'Ghi chú']

In [13]:
# Đếm số lượng giá trị duy nhất cho mỗi cột
for column in object_columns: 
    print("*, Column: ", column)   
    print(len(sh_channel[column].unique()), "unique values", "\t & \t", 
          sh_channel[column].notnull().sum(), "non-null values\t", 
          round(100* sh_channel[column].notnull().sum()/len(sh_channel[column]),2), 
          "% non-null") 

*, Column:  Thương hiệu
17 unique values 	 & 	 3589 non-null values	 100.0 % non-null
*, Column:  Ngành hàng
16 unique values 	 & 	 3589 non-null values	 100.0 % non-null
*, Column:  Sản phẩm
274 unique values 	 & 	 3589 non-null values	 100.0 % non-null
*, Column:  Ghi chú
6 unique values 	 & 	 5 non-null values	 0.14 % non-null


In [14]:
for column in object_columns:
    print('-'*100)
    print("*, Column: ", column)
    print(sh_channel[column].value_counts())
    print('-'*100)

----------------------------------------------------------------------------------------------------
*, Column:  Thương hiệu
Vinamilk        947
Nutifood        396
Nutricare       392
TH True Milk    375
Vitadairy       289
Nestlé          268
FCV             190
IDP             152
Mead Johnson    130
Abbott          108
Vinasoy          70
Đà Lạt Milk      62
Hipp             54
Fonterra         54
Betagen          42
Yakult           36
Hoff             24
Name: Thương hiệu, dtype: int64
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
*, Column:  Ngành hàng
SB       1377
SN        588
SC        463
BRAND     284
NGK       232
SĐN       209
SĐ        124
KEM       118
BAD        72
BDD        50
ĐƯỜNG      19
BƠ         18
PM         17
FOOD       12
TUNL        5
Brand       1
Name: Ngành hàng, dtype: int64
-----------------------

Sheet channel.sentiment

In [15]:
sh_channel.sentiment.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7412 entries, 0 to 7411
Data columns (total 37 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   Thương hiệu                   7412 non-null   object        
 1   Ngành hàng                    7412 non-null   object        
 2   Sản phẩm                      7412 non-null   object        
 3   Thời gian                     7412 non-null   datetime64[ns]
 4   Sentiment                     7412 non-null   object        
 5   Fanpage                       4205 non-null   object        
 6   FB page                       4332 non-null   float64       
 7   FB user                       4271 non-null   float64       
 8   FB group                      5041 non-null   float64       
 9   Tiktok                        2681 non-null   float64       
 10  Forum                         831 non-null    float64       
 11  News                          

In [16]:
object_columns = list(sh_channel.sentiment.select_dtypes(['object']).columns)
object_columns

['Thương hiệu',
 'Ngành hàng',
 'Sản phẩm',
 'Sentiment',
 'Fanpage',
 'Ghi chú',
 'Phân loại']

In [17]:
# Đếm số lượng giá trị duy nhất cho mỗi cột
for column in object_columns: 
    print("*, Column: ", column)   
    print(len(sh_channel.sentiment[column].unique()), "unique values", "\t & \t", 
          sh_channel.sentiment[column].notnull().sum(), "non-null values\t", 
          round(100* sh_channel.sentiment[column].notnull().sum()/len(sh_channel.sentiment[column]),2), 
          "% non-null") 

*, Column:  Thương hiệu
17 unique values 	 & 	 7412 non-null values	 100.0 % non-null
*, Column:  Ngành hàng
16 unique values 	 & 	 7412 non-null values	 100.0 % non-null
*, Column:  Sản phẩm
276 unique values 	 & 	 7412 non-null values	 100.0 % non-null
*, Column:  Sentiment
3 unique values 	 & 	 7412 non-null values	 100.0 % non-null
*, Column:  Fanpage
1004 unique values 	 & 	 4205 non-null values	 56.73 % non-null
*, Column:  Ghi chú
2 unique values 	 & 	 3 non-null values	 0.04 % non-null
*, Column:  Phân loại
19 unique values 	 & 	 7304 non-null values	 98.54 % non-null


In [18]:
for column in object_columns:
    print('-'*100)
    print("*, Column: ", column)
    print(sh_channel.sentiment[column].value_counts())
    print('-'*100)

----------------------------------------------------------------------------------------------------
*, Column:  Thương hiệu
Vinamilk        1971
TH True Milk     880
Nutifood         687
Nutricare        620
Vitadairy        576
Nestlé           529
FCV              391
IDP              341
Abbott           320
Mead Johnson     295
Vinasoy          167
Hipp             141
Đà Lạt Milk      135
Fonterra         115
Yakult            97
Betagen           84
Hoff              63
Name: Thương hiệu, dtype: int64
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
*, Column:  Ngành hàng
SB       2787
SN       1314
SC       1002
BRAND     650
NGK       431
SĐN       412
SĐ        230
KEM       221
BAD       132
BDD        94
BƠ         40
PM         30
FOOD       25
ĐƯỜNG      24
Brand      13
TUNL        7
Name: Ngành hàng, dtype: int64
------

# Tổng quan

In [6]:
year = ytd= "2024"
pre_ytd = ['01/2023','02/2023','03/2023','04/2023','05/2023','06/2023']
Q = ['04/2024','05/2024','06/2024']
#Q1=
#Q2=
#Q3=

In [7]:
exclude_brands = ["Hoff", "Betagen", "Yakult"]

# YTD 2023 và Quý 2024

YTD 2023

In [21]:
#Slide 8 Thị phần ngành sữa YTD 2023
filtered_sh_sentiment_q = sh_sentiment[(sh_sentiment["Thời gian"].isin(pre_ytd)) & ~sh_sentiment["Thương hiệu"].isin(exclude_brands)]
SOV_pre = filtered_sh_sentiment_q.groupby(["Thương hiệu"]).agg({"Total new": "sum"}).sort_values(by="Total new",ascending=False).reset_index().round()
SOV_pre = SOV_pre.append(pd.DataFrame({"Thương hiệu": ["Total"], "Total new": SOV_pre["Total new"].sum()}), ignore_index=True)
SOV_pre

C:\Users\Kompa-KTelt056\AppData\Local\Temp\ipykernel_8776\645784089.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  SOV_pre = SOV_pre.append(pd.DataFrame({"Thương hiệu": ["Total"], "Total new": SOV_pre["Total new"].sum()}), ignore_index=True)


,Thương hiệu,Total new
0,Vinamilk,1773134.0
1,Nestlé,748705.0
2,Abbott,602213.0
3,Nutifood,399953.0
4,FCV,377402.0
5,Mead Johnson,120666.0
6,IDP,117208.0
7,TH True Milk,100580.0
8,Vinasoy,91156.0
9,Vitadairy,63927.0


Quý 2024

In [23]:
filtered_sh_sentiment_q = sh_sentiment[(sh_sentiment["Thời gian"].isin(Q)) & ~sh_sentiment["Thương hiệu"].isin(exclude_brands)]
SOV_Q = filtered_sh_sentiment_q.groupby(["Thương hiệu"]).agg({"Total new": "sum"}).sort_values(by="Total new",ascending=False).reset_index().round()
SOV_Q = SOV_Q.append(pd.DataFrame({"Thương hiệu": ["Total"], "Total new": SOV_Q["Total new"].sum()}), ignore_index=True)
SOV_Q

C:\Users\Kompa-KTelt056\AppData\Local\Temp\ipykernel_8776\1540900838.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  SOV_Q = SOV_Q.append(pd.DataFrame({"Thương hiệu": ["Total"], "Total new": SOV_Q["Total new"].sum()}), ignore_index=True)


,Thương hiệu,Total new
0,Vinamilk,2533920.0
1,Nestlé,2260880.0
2,Nutifood,1598263.0
3,Abbott,917307.0
4,TH True Milk,245063.0
5,IDP,165904.0
6,FCV,164623.0
7,Vitadairy,161237.0
8,Fonterra,111832.0
9,Vinasoy,96661.0


In [24]:
with pd.ExcelWriter(r"C:\Users\Kompa-KTelt056\Desktop\Data Kompa\Vinamilk\final.xlsx", engine='openpyxl',mode='a') as writer:
    SOV_Q.to_excel(writer, index=False, sheet_name='Slide 24')

In [25]:
filtered_sh_sentiment = sh_sentiment[(sh_sentiment["Thời gian"].isin(Q)) & ~sh_sentiment["Thương hiệu"].isin(exclude_brands)]
brand_sent_Q = filtered_sh_sentiment.groupby(["Thương hiệu"]).agg({"Positive new": "sum","Neutral new": "sum","Negative new": "sum","Total new": "sum"}).sort_values(by="Total new",ascending=False).reset_index().round(decimals=2)
brand_sent_Q["SOV"] = brand_sent_Q["Total new"] / brand_sent_Q["Total new"].sum()
brand_sent_Q.set_index('Thương hiệu', inplace = True)
brand_sent_Q = brand_sent_Q.T.round(decimals=3)
brand_sent_Q['Total'] = brand_sent_Q.sum(axis=1)
brand_sent_Q.reset_index(inplace=True)
brand_sent_Q

Thương hiệu,index,Vinamilk,Nestlé,Nutifood,Abbott,TH True Milk,IDP,FCV,Vitadairy,Fonterra,Vinasoy,Mead Johnson,Hipp,Nutricare,Đà Lạt Milk,Total
0,Positive new,2019349.000,2197066.450,1530084.04,596896.280,32123.310,102492.53,35634.39,69030.990,97299.000,66123.900,36177.33,11853.280,5314.000,402.200,6799846.70
1,Neutral new,513757.000,63600.000,67868.96,319568.550,212007.710,61779.96,128759.95,92017.530,14527.000,30516.100,44854.67,41902.060,22707.000,11705.820,1625572.31
2,Negative new,814.000,213.550,310.01,842.160,931.990,1631.51,228.67,188.480,6.000,21.000,413.00,129.660,29.000,7.970,5767.00
3,Total new,2533920.000,2260880.000,1598263.00,917307.000,245063.000,165904.00,164623.00,161237.000,111832.000,96661.000,81445.00,53885.000,28050.000,12116.000,8431186.00
4,SOV,0.301,0.268,0.19,0.109,0.029,0.02,0.02,0.019,0.013,0.011,0.01,0.006,0.003,0.001,1.00


In [26]:
with pd.ExcelWriter(r"C:\Users\Kompa-KTelt056\Desktop\Data Kompa\Vinamilk\final.xlsx", engine='openpyxl',mode='a') as writer:
    brand_sent_Q.to_excel(writer, index=False, sheet_name='Slide 25')

In [27]:
#Thuộc tính sản phẩm TYD
filtered_sh_label = sh_label[(sh_label["Thời gian"].isin(Q)) & ~sh_label["Thương hiệu"].isin(exclude_brands)]
Label_Q = filtered_sh_label.groupby(["Thương hiệu"]).agg({"Bao bì2": "sum","Chất lượng2": "sum","Công dụng2": "sum","Giá2": "sum","Hương vị2": "sum","Khuyến mãi2": "sum","Thành phần2": "sum","Trải nghiệm SD2": "sum","Dị vật2": "sum","Total new": "sum"}).sort_values(by="Total new",ascending=False).reset_index().round()
column_sums = Label_Q.iloc[:, 1:].sum()
Label_Q.loc['Total'] = ['Total'] + column_sums.tolist()
Label_Q['Total Labels'] = Label_Q.sum(axis=1) 
Label_Q['Total Labels'] = Label_Q['Total Labels'] - Label_Q['Total new']
Label_Q.set_index('Thương hiệu', inplace = True)
Label_Q = Label_Q.transpose().sort_values(by='Total',ascending=False)
Label_Q = Label_Q.drop('Total new') 
Label_Q["SOV"] = Label_Q["Total"] / Label_Q.loc["Total Labels","Total"]
Label_Q.index = ['Total Labels','Công dụng2','Trải nghiệm SD2','Hương vị2','Thành phần2','Chất lượng2','Khuyến mãi2','Giá2','Bao bì2','Dị vật2']
Label_Q.reset_index(inplace=True)
Label_Q
#80%

C:\Users\Kompa-KTelt056\AppData\Local\Temp\ipykernel_8776\647383585.py:6: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  Label_Q['Total Labels'] = Label_Q.sum(axis=1)


Thương hiệu,index,Vinamilk,Nestlé,Nutifood,Abbott,TH True Milk,IDP,FCV,Vitadairy,Fonterra,Vinasoy,Mead Johnson,Hipp,Nutricare,Đà Lạt Milk,Total,SOV
0,Total Labels,1558131.0,868916.0,300281.0,298394.0,47158.0,22751.0,27124.0,74342.0,7821.0,3938.0,52227.0,40820.0,11795.0,1318.0,3315016.0,1.000000
1,Công dụng2,781927.0,765688.0,278443.0,159046.0,4497.0,959.0,20328.0,35056.0,3042.0,2188.0,17182.0,11999.0,8769.0,179.0,2089303.0,0.630254
2,Trải nghiệm SD2,443205.0,68918.0,615.0,66830.0,4453.0,286.0,4493.0,7083.0,2860.0,946.0,7782.0,8923.0,1563.0,832.0,618789.0,0.186662
3,Hương vị2,161172.0,7943.0,910.0,11833.0,5441.0,6001.0,789.0,2386.0,393.0,349.0,2596.0,4652.0,355.0,239.0,205059.0,0.061858
4,Thành phần2,45935.0,13340.0,1952.0,29295.0,28601.0,5575.0,832.0,24720.0,1004.0,82.0,10334.0,5141.0,158.0,13.0,166982.0,0.050371
5,Chất lượng2,61523.0,1596.0,7990.0,4922.0,910.0,314.0,55.0,914.0,129.0,12.0,763.0,5625.0,119.0,7.0,84879.0,0.025604
6,Khuyến mãi2,54166.0,7798.0,657.0,11916.0,167.0,1443.0,238.0,1267.0,156.0,118.0,4337.0,2423.0,38.0,2.0,84726.0,0.025558
7,Giá2,6199.0,926.0,5891.0,13607.0,3027.0,5196.0,379.0,2902.0,214.0,99.0,4612.0,2016.0,737.0,46.0,45851.0,0.013831
8,Bao bì2,4003.0,2707.0,3823.0,945.0,48.0,2977.0,10.0,14.0,23.0,144.0,4621.0,41.0,56.0,0.0,19412.0,0.005856
9,Dị vật2,1.0,0.0,0.0,0.0,14.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15.0,0.000005


In [28]:
with pd.ExcelWriter(r"C:\Users\Kompa-KTelt056\Desktop\Data Kompa\Vinamilk\final.xlsx", engine='openpyxl',mode='a') as writer:
    Label_Q.to_excel(writer, index=False, sheet_name='Slide 26 - 27')

In [29]:
#Thảo luận qua các kênh TYD
filtered_sh_channel = sh_channel[(sh_channel["Thời gian"].isin(Q)) & ~sh_channel["Thương hiệu"].isin(exclude_brands)]
brand_channel_Q = filtered_sh_channel.groupby(["Thương hiệu"]).agg({"Total Facebook": "sum","Tiktok new": "sum","Youtube new": "sum","Forum new": "sum","News new": "sum","Total new": "sum"}).sort_values(by="Total new",ascending=False).reset_index().round()
brand_channel_Q = brand_channel_Q.append(pd.DataFrame({"Thương hiệu": ["Total"], "Total new": brand_channel_Q["Total new"].sum(),"Tiktok new": brand_channel_Q["Tiktok new"].sum(),"Total Facebook": brand_channel_Q["Total Facebook"].sum(),"Forum new": brand_channel_Q["Forum new"].sum(),"News new": brand_channel_Q["News new"].sum(),"Youtube new": brand_channel_Q["Youtube new"].sum()}), ignore_index=True)
brand_channel_Q['Khác'] = brand_channel_Q['Total new'] - brand_channel_Q[['Total Facebook', 'Tiktok new', 'Forum new', 'News new', 'Youtube new']].sum(axis=1) 
brand_channel_Q[['Thương hiệu','Total Facebook', 'Tiktok new', 'Youtube new', 'Forum new', 'News new','Khác','Total new']]

C:\Users\Kompa-KTelt056\AppData\Local\Temp\ipykernel_8776\411848611.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  brand_channel_Q = brand_channel_Q.append(pd.DataFrame({"Thương hiệu": ["Total"], "Total new": brand_channel_Q["Total new"].sum(),"Tiktok new": brand_channel_Q["Tiktok new"].sum(),"Total Facebook": brand_channel_Q["Total Facebook"].sum(),"Forum new": brand_channel_Q["Forum new"].sum(),"News new": brand_channel_Q["News new"].sum(),"Youtube new": brand_channel_Q["Youtube new"].sum()}), ignore_index=True)


,Thương hiệu,Total Facebook,Tiktok new,Youtube new,Forum new,News new,Khác,Total new
0,Vinamilk,2429607.0,80840.0,8820,6458,6916,1279.0,2533920.0
1,Nestlé,2244562.0,14988.0,66,125,905,234.0,2260880.0
2,Nutifood,1583887.0,12990.0,307,11,451,617.0,1598263.0
3,Abbott,912318.0,4719.0,14,162,87,7.0,917307.0
4,TH True Milk,236907.0,5321.0,1234,117,1419,65.0,245063.0
5,IDP,116302.0,44977.0,4271,68,192,94.0,165904.0
6,FCV,137217.0,27050.0,213,40,79,24.0,164623.0
7,Vitadairy,154934.0,5920.0,108,19,70,186.0,161237.0
8,Fonterra,111311.0,476.0,8,4,31,2.0,111832.0
9,Vinasoy,94818.0,1513.0,179,11,124,16.0,96661.0


In [30]:
with pd.ExcelWriter(r"C:\Users\Kompa-KTelt056\Desktop\Data Kompa\Vinamilk\final.xlsx", engine='openpyxl',mode='a') as writer:
    brand_channel_Q.to_excel(writer, index=False, sheet_name='Slide 28')

# YTD 2024

In [9]:
sh_relevant["Thời gian"] = sh_relevant["Thời gian"].astype(str)
sh_sentiment["Thời gian"] = sh_sentiment["Thời gian"].astype(str)
sh_channel["Thời gian"] = sh_channel["Thời gian"].astype(str)
sh_label["Thời gian"] = sh_label["Thời gian"].astype(str)
sh_channel.sentiment["Thời gian"] = sh_channel.sentiment["Thời gian"].astype(str)

In [32]:
#Slide 8 Thị phần ngành sữa YTD 2024
filtered_sh_sentiment_q = sh_sentiment[(sh_sentiment["Thời gian"].str.contains(year,na=False)) & ~sh_sentiment["Thương hiệu"].isin(exclude_brands)]
SOV = filtered_sh_sentiment_q.groupby(["Thương hiệu"]).agg({"Total new": "sum"}).sort_values(by="Total new",ascending=False).reset_index().round()
SOV = SOV.append(pd.DataFrame({"Thương hiệu": ["Total"], "Total new": SOV["Total new"].sum()}), ignore_index=True)
SOV

C:\Users\Kompa-KTelt056\AppData\Local\Temp\ipykernel_8776\4107313346.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  SOV = SOV.append(pd.DataFrame({"Thương hiệu": ["Total"], "Total new": SOV["Total new"].sum()}), ignore_index=True)


,Thương hiệu,Total new
0,Vinamilk,5503090.0
1,Nestlé,3209541.0
2,Nutifood,2750664.0
3,Abbott,1197049.0
4,IDP,1004974.0
5,TH True Milk,341939.0
6,FCV,272428.0
7,Mead Johnson,263176.0
8,Vitadairy,250207.0
9,Fonterra,195394.0


In [33]:
#Silde 9
df = pd.merge(SOV, SOV_pre, on='Thương hiệu', how='left')
df['% growth'] = df['Total new_x']/df['Total new_y']-1
df = df.rename(columns={'Total new_x': 'YTD 2024'})
df = df.rename(columns={'Total new_y': 'Q 2023'})
df

,Thương hiệu,YTD 2024,Q 2023,% growth
0,Vinamilk,5503090.0,1773134.0,2.103595
1,Nestlé,3209541.0,748705.0,3.286790
2,Nutifood,2750664.0,399953.0,5.877468
3,Abbott,1197049.0,602213.0,0.987750
4,IDP,1004974.0,117208.0,7.574278
5,TH True Milk,341939.0,100580.0,2.399672
6,FCV,272428.0,377402.0,-0.278149
7,Mead Johnson,263176.0,120666.0,1.181029
8,Vitadairy,250207.0,63927.0,2.913949
9,Fonterra,195394.0,7957.0,23.556240


In [34]:
with pd.ExcelWriter(r"C:\Users\Kompa-KTelt056\Desktop\Data Kompa\Vinamilk\final.xlsx", engine='openpyxl',mode='a') as writer:
    df.to_excel(writer, index=False, sheet_name='Slide 8-9')

In [35]:
#Sentiment các brands YTD
filtered_sh_sentiment = sh_sentiment[(sh_sentiment["Thời gian"].str.contains(year,na=False)) & ~sh_sentiment["Thương hiệu"].isin(exclude_brands)]
brand_sent = filtered_sh_sentiment.groupby(["Thương hiệu"]).agg({"Positive new": "sum","Neutral new": "sum","Negative new": "sum","Total new": "sum"}).sort_values(by="Total new",ascending=False).reset_index().round(decimals=2)
brand_sent["SOV"] = brand_sent["Total new"] / brand_sent["Total new"].sum()
brand_sent.set_index('Thương hiệu', inplace = True)
brand_sent = brand_sent.T.round(decimals=3)
brand_sent['Total'] = brand_sent.sum(axis=1)
brand_sent.reset_index(inplace=True)
brand_sent

Thương hiệu,index,Vinamilk,Nestlé,Nutifood,Abbott,IDP,TH True Milk,FCV,Mead Johnson,Vitadairy,Fonterra,Vinasoy,Hipp,Nutricare,Đà Lạt Milk,Total
0,Positive new,4198483.00,3030287.07,2625424.19,712343.610,877448.670,47453.730,53786.720,177880.500,98576.330,122262.880,87908.06,16823.350,7019.000,638.160,1.205634e+07
1,Neutral new,1303370.00,178894.96,124749.18,470559.820,122930.720,292752.090,218175.960,84769.090,151409.010,73061.610,60506.11,60539.750,40753.000,18680.870,3.201152e+06
2,Negative new,1237.00,358.97,490.63,14145.570,4594.610,1733.190,465.320,526.410,221.660,69.520,222.83,150.900,54.000,10.970,2.428158e+04
3,Total new,5503090.00,3209541.00,2750664.00,1197049.000,1004974.000,341939.000,272428.000,263176.000,250207.000,195394.000,148637.00,77514.000,47826.000,19330.000,1.528177e+07
4,SOV,0.36,0.21,0.18,0.078,0.066,0.022,0.018,0.017,0.016,0.013,0.01,0.005,0.003,0.001,9.990000e-01


In [36]:
with pd.ExcelWriter(r"C:\Users\Kompa-KTelt056\Desktop\Data Kompa\Vinamilk\final.xlsx", engine='openpyxl',mode='a') as writer:
    brand_sent.to_excel(writer, index=False, sheet_name='Silde 10')

In [37]:
#Thuộc tính sản phẩm TYD
filtered_sh_label = sh_label[(sh_label["Thời gian"].str.contains(year,na=False)) & ~sh_label["Thương hiệu"].isin(exclude_brands)]
Label = filtered_sh_label.groupby(["Thương hiệu"]).agg({"Bao bì2": "sum","Chất lượng2": "sum","Công dụng2": "sum","Giá2": "sum","Hương vị2": "sum","Khuyến mãi2": "sum","Thành phần2": "sum","Trải nghiệm SD2": "sum","Dị vật2": "sum","Total new": "sum"}).sort_values(by="Total new",ascending=False).reset_index().round()
column_sums = Label.iloc[:, 1:].sum()
Label.loc['Total'] = ['Total'] + column_sums.tolist()
Label['Total Labels'] = Label.sum(axis=1) 
Label['Total Labels'] = Label['Total Labels'] - Label['Total new']
Label.set_index('Thương hiệu', inplace = True)
Label = Label.transpose().sort_values(by='Total',ascending=False)
Label = Label.drop('Total new') 
Label["SOV"] = Label["Total"] / Label.loc["Total Labels","Total"]
Label.index = ['Total Labels','Công dụng2','Trải nghiệm SD2','Hương vị2','Thành phần2','Chất lượng2','Khuyến mãi2','Giá2','Bao bì2','Dị vật2']
Label.reset_index(inplace=True)
Label
#80%

C:\Users\Kompa-KTelt056\AppData\Local\Temp\ipykernel_8776\3884440672.py:6: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  Label['Total Labels'] = Label.sum(axis=1)


Thương hiệu,index,Vinamilk,Nestlé,Nutifood,Abbott,IDP,TH True Milk,FCV,Mead Johnson,Vitadairy,Fonterra,Vinasoy,Hipp,Nutricare,Đà Lạt Milk,Total,SOV
0,Total Labels,3869875.0,946261.0,883154.0,461076.0,34494.0,85710.0,47293.0,84354.0,124692.0,18535.0,6246.0,56105.0,17138.0,1990.0,6636923.0,1.000000
1,Công dụng2,1725768.0,784940.0,651257.0,231432.0,2978.0,6449.0,36168.0,26780.0,58284.0,6914.0,3220.0,16205.0,12664.0,194.0,3563253.0,0.536883
2,Trải nghiệm SD2,1241311.0,118212.0,26422.0,108919.0,1131.0,26024.0,7696.0,12615.0,14076.0,6441.0,1530.0,15435.0,2547.0,1247.0,1583606.0,0.238605
3,Hương vị2,494573.0,11735.0,6636.0,19751.0,10599.0,5959.0,1148.0,5244.0,3288.0,695.0,448.0,5384.0,443.0,269.0,566172.0,0.085306
4,Thành phần2,127927.0,13759.0,30380.0,37442.0,6664.0,39822.0,1104.0,13038.0,41711.0,2036.0,230.0,6740.0,244.0,174.0,321271.0,0.048407
5,Chất lượng2,105703.0,10884.0,90601.0,17741.0,1671.0,696.0,238.0,6788.0,2388.0,1034.0,164.0,3690.0,65.0,5.0,241668.0,0.036413
6,Khuyến mãi2,132960.0,2761.0,65602.0,24849.0,1033.0,1729.0,83.0,1231.0,1700.0,511.0,295.0,6568.0,209.0,15.0,239546.0,0.036093
7,Giá2,24636.0,1134.0,7617.0,19816.0,6764.0,4814.0,846.0,5879.0,3221.0,313.0,208.0,2040.0,892.0,84.0,78264.0,0.011792
8,Bao bì2,16847.0,2836.0,4639.0,1126.0,3654.0,203.0,10.0,12779.0,24.0,591.0,151.0,43.0,74.0,2.0,42979.0,0.006476
9,Dị vật2,150.0,0.0,0.0,0.0,0.0,14.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,164.0,0.000025


In [38]:
with pd.ExcelWriter(r"C:\Users\Kompa-KTelt056\Desktop\Data Kompa\Vinamilk\final.xlsx", engine='openpyxl',mode='a') as writer:
    Label.to_excel(writer, index=False, sheet_name='Slide 11 - 12')

In [39]:
#Thảo luận qua các kênh TYD
filtered_sh_channel = sh_channel[(sh_channel["Thời gian"].str.contains(year,na=False)) & ~sh_channel["Thương hiệu"].isin(exclude_brands)]
brand_channel = filtered_sh_channel.groupby(["Thương hiệu"]).agg({"Total Facebook": "sum","Tiktok new": "sum","Youtube new": "sum","Forum new": "sum","News new": "sum","Total new": "sum"}).sort_values(by="Total new",ascending=False).reset_index().round()
brand_channel = brand_channel.append(pd.DataFrame({"Thương hiệu": ["Total"], "Total new": brand_channel["Total new"].sum(),"Tiktok new": brand_channel["Tiktok new"].sum(),"Total Facebook": brand_channel["Total Facebook"].sum(),"Forum new": brand_channel["Forum new"].sum(),"News new": brand_channel["News new"].sum(),"Youtube new": brand_channel["Youtube new"].sum()}), ignore_index=True)
brand_channel['Khác'] = brand_channel['Total new'] - brand_channel[['Total Facebook', 'Tiktok new', 'Forum new', 'News new', 'Youtube new']].sum(axis=1) 
brand_channel[['Thương hiệu','Total Facebook', 'Tiktok new', 'Youtube new', 'Forum new', 'News new','Khác','Total new']]

C:\Users\Kompa-KTelt056\AppData\Local\Temp\ipykernel_8776\3069266607.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  brand_channel = brand_channel.append(pd.DataFrame({"Thương hiệu": ["Total"], "Total new": brand_channel["Total new"].sum(),"Tiktok new": brand_channel["Tiktok new"].sum(),"Total Facebook": brand_channel["Total Facebook"].sum(),"Forum new": brand_channel["Forum new"].sum(),"News new": brand_channel["News new"].sum(),"Youtube new": brand_channel["Youtube new"].sum()}), ignore_index=True)


,Thương hiệu,Total Facebook,Tiktok new,Youtube new,Forum new,News new,Khác,Total new
0,Vinamilk,5268468.0,197849.0,12495,12127,10775,1376.0,5503090.0
1,Nestlé,3187980.0,19107.0,146,210,1243,855.0,3209541.0
2,Nutifood,2724883.0,23675.0,365,21,572,1148.0,2750664.0
3,Abbott,1181745.0,14727.0,71,304,194,8.0,1197049.0
4,IDP,711132.0,285355.0,6988,105,487,907.0,1004974.0
5,TH True Milk,321444.0,17092.0,1336,326,1675,66.0,341939.0
6,FCV,240915.0,30882.0,394,68,133,36.0,272428.0
7,Mead Johnson,257705.0,5096.0,58,67,102,148.0,263176.0
8,Vitadairy,240400.0,9053.0,257,45,92,360.0,250207.0
9,Fonterra,190072.0,5231.0,34,6,49,2.0,195394.0


In [40]:
with pd.ExcelWriter(r"C:\Users\Kompa-KTelt056\Desktop\Data Kompa\Vinamilk\final.xlsx", engine='openpyxl',mode='a') as writer:
    brand_channel.to_excel(writer, index=False, sheet_name='Slide 13')

## Thị phần thảo luận ngành sữa qua các kênh YTD

In [41]:
#Facebook
filtered_sh_channel.sentiment = sh_channel.sentiment[(sh_channel.sentiment["Thời gian"].str.contains(year,na=False)) & ~sh_channel.sentiment["Thương hiệu"].isin(exclude_brands)]
fb = filtered_sh_channel.sentiment.pivot_table(index='Thương hiệu', columns='Sentiment', values='Facebook', aggfunc='sum').round(decimals=0)
fb.reset_index(inplace=True)
fb['Total'] = fb.sum(axis=1)
fb = fb.sort_values(by = 'Total', ascending = False)
fb

C:\Users\Kompa-KTelt056\AppData\Local\Temp\ipykernel_8776\3508456112.py:2: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  filtered_sh_channel.sentiment = sh_channel.sentiment[(sh_channel.sentiment["Thời gian"].str.contains(year,na=False)) & ~sh_channel.sentiment["Thương hiệu"].isin(exclude_brands)]
C:\Users\Kompa-KTelt056\AppData\Local\Temp\ipykernel_8776\3508456112.py:5: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  fb['Total'] = fb.sum(axis=1)


Sentiment,Thương hiệu,Negative,Neutral,Positive,Total
10,Vinamilk,783.0,1187314.0,4075770.0,5263867.0
6,Nestlé,211.0,176839.0,3028043.0,3205093.0
7,Nutifood,268.0,114108.0,2620904.0,2735280.0
0,Abbott,11300.0,452085.0,710160.0,1173545.0
4,IDP,3842.0,102829.0,640335.0,747006.0
9,TH True Milk,1478.0,273246.0,45969.0,320693.0
5,Mead Johnson,423.0,82984.0,176480.0,259887.0
1,FCV,403.0,205879.0,35763.0,242045.0
12,Vitadairy,198.0,146954.0,94451.0,241603.0
2,Fonterra,69.0,69752.0,120244.0,190065.0


In [42]:
with pd.ExcelWriter(r"C:\Users\Kompa-KTelt056\Desktop\Data Kompa\Vinamilk\final.xlsx", engine='openpyxl',mode='a') as writer:
    fb.to_excel(writer, index=False, sheet_name='Slide 14 - Facebook')

In [43]:
#Tiktok
filtered_sh_channel.sentiment = sh_channel.sentiment[(sh_channel.sentiment["Thời gian"].str.contains(year,na=False)) & ~sh_channel.sentiment["Thương hiệu"].isin(exclude_brands)]
tiktok = filtered_sh_channel.sentiment.pivot_table(index='Thương hiệu', columns='Sentiment', values='Tiktok new', aggfunc='sum').round(decimals=0)
tiktok.reset_index(inplace=True)
tiktok['Total'] = tiktok.sum(axis=1)
tiktok = tiktok.sort_values(by = 'Total', ascending = False)
tiktok

C:\Users\Kompa-KTelt056\AppData\Local\Temp\ipykernel_8776\945582446.py:5: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  tiktok['Total'] = tiktok.sum(axis=1)


Sentiment,Thương hiệu,Negative,Neutral,Positive,Total
4,IDP,730.0,17967.0,235633.0,254330.0
10,Vinamilk,327.0,88392.0,113731.0,202450.0
1,FCV,58.0,11795.0,17998.0,29851.0
0,Abbott,2765.0,18070.0,2124.0,22959.0
9,TH True Milk,245.0,18307.0,1242.0,19794.0
7,Nutifood,219.0,10496.0,3975.0,14690.0
12,Vitadairy,17.0,4158.0,3840.0,8015.0
8,Nutricare,8.0,2757.0,3220.0,5985.0
2,Fonterra,0.0,3236.0,2003.0,5239.0
6,Nestlé,114.0,1633.0,1336.0,3083.0


In [44]:
with pd.ExcelWriter(r"C:\Users\Kompa-KTelt056\Desktop\Data Kompa\Vinamilk\final.xlsx", engine='openpyxl',mode='a') as writer:
    tiktok.to_excel(writer, index=False, sheet_name='Slide 15 - Tiktok')

In [45]:
#News
filtered_sh_channel.sentiment = sh_channel.sentiment[(sh_channel.sentiment["Thời gian"].str.contains(year,na=False)) & ~sh_channel.sentiment["Thương hiệu"].isin(exclude_brands)]
news = filtered_sh_channel.sentiment.pivot_table(index='Thương hiệu', columns='Sentiment', values='News new', aggfunc='sum').round(decimals=0)
news.reset_index(inplace=True)
news['Total'] = news.sum(axis=1)
news = news.sort_values(by = 'Total', ascending = False)
news

C:\Users\Kompa-KTelt056\AppData\Local\Temp\ipykernel_8776\3519821983.py:5: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  news['Total'] = news.sum(axis=1)


Sentiment,Thương hiệu,Negative,Neutral,Positive,Total
10,Vinamilk,1,8672,2102,10775
6,Nestlé,11,330,146,487
9,TH True Milk,0,257,189,446
11,Vinasoy,0,188,34,222
4,IDP,2,149,69,220
0,Abbott,71,123,0,194
7,Nutifood,0,72,110,182
1,FCV,0,124,2,126
12,Vitadairy,0,89,3,92
13,Đà Lạt Milk,0,85,0,85


In [46]:
with pd.ExcelWriter(r"C:\Users\Kompa-KTelt056\Desktop\Data Kompa\Vinamilk\final.xlsx", engine='openpyxl',mode='a') as writer:
    news.to_excel(writer, index=False, sheet_name='Slide 16 - News')

In [47]:
#Youtube
filtered_sh_channel.sentiment = sh_channel.sentiment[(sh_channel.sentiment["Thời gian"].str.contains(year,na=False)) & ~sh_channel.sentiment["Thương hiệu"].isin(exclude_brands)]
youtube = filtered_sh_channel.sentiment.pivot_table(index='Thương hiệu', columns='Sentiment', values='Youtube new', aggfunc='sum').round(decimals=0)
youtube.reset_index(inplace=True)
youtube['Total'] = youtube.sum(axis=1)
youtube = youtube.sort_values(by = 'Total', ascending = False)
youtube

C:\Users\Kompa-KTelt056\AppData\Local\Temp\ipykernel_8776\152311339.py:5: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  youtube['Total'] = youtube.sum(axis=1)


Sentiment,Thương hiệu,Negative,Neutral,Positive,Total
10,Vinamilk,8,7155,5332,12495
4,IDP,15,1941,550,2506
9,TH True Milk,2,816,14,832
1,FCV,0,323,8,331
11,Vinasoy,0,204,19,223
12,Vitadairy,1,153,25,179
8,Nutricare,0,128,7,135
0,Abbott,0,35,34,69
6,Nestlé,4,37,4,45
7,Nutifood,0,25,16,41


In [48]:
with pd.ExcelWriter(r"C:\Users\Kompa-KTelt056\Desktop\Data Kompa\Vinamilk\final.xlsx", engine='openpyxl',mode='a') as writer:
    youtube.to_excel(writer, index=False, sheet_name='Slide 17 - Youtube')

# Ngành hàng YTD

SỮA NƯỚC

In [15]:
filtered_sh_sentiment = sh_sentiment[(sh_sentiment["Thời gian"].str.contains(year,na=False)) & ~sh_sentiment["Thương hiệu"].isin(exclude_brands)]
SN = filtered_sh_sentiment.pivot_table(index='Thời gian', columns='Thương hiệu', values='Total new', aggfunc='sum').round(decimals=0)
#SN = SN.sort_values(by = 'Total new', ascending = False)
SN

Thương hiệu,Abbott,FCV,Fonterra,Hipp,IDP,Mead Johnson,Nestlé,Nutifood,Nutricare,TH True Milk,Vinamilk,Vinasoy,Vitadairy,Đà Lạt Milk
Thời gian,,,,,,,,,,,,,,
2024-01-01,157165.0,33599.0,77607.0,6251.0,18692.0,18280.0,395300.0,50854.0,6869.0,18513.0,975475.0,25113.0,35467.0,2098.0
2024-02-01,69971.0,22038.0,1991.0,3744.0,34573.0,16414.0,262527.0,37496.0,4878.0,14599.0,1098439.0,9279.0,17287.0,2149.0
2024-03-01,52606.0,52168.0,3964.0,13634.0,785805.0,147037.0,290834.0,1064051.0,8029.0,63764.0,895256.0,17584.0,36216.0,2967.0
2024-04-01,106687.0,51514.0,62028.0,21098.0,75570.0,40747.0,1257711.0,664963.0,9878.0,78560.0,493261.0,7316.0,30947.0,3933.0
2024-05-01,106023.0,66290.0,38271.0,17744.0,35512.0,19785.0,599086.0,321144.0,6743.0,80097.0,1086483.0,14736.0,43235.0,2504.0
2024-06-01,704597.0,46819.0,11533.0,15043.0,54822.0,20913.0,404083.0,612156.0,11429.0,86406.0,954176.0,74609.0,87055.0,5679.0
